In [2]:
import pandas as pd
import os

# Court Documents

In [ ]:
import pandas as pd

opinion_df_all = pd.read_csv("../court_documents/unprocessed_appeals_plain_text__current.csv", index_col=0)
long_cases = opinion_df_all.loc[lambda df: df['plain_text'].apply(lambda x: len(x.split()) > 400)]

def split_text_head(text):
    split_points = [
        'OPINION', 'FACTS', 'CHARGE', 'PROCEDURAL BACKGROUND',
        'INTRODUCTION', 'BACKGROUND', 'I. Facts', 'FACT', ', Judge', ', Presiding Judge', ', Chief Judge', 
    ]
    
    for point in split_points:
        if 'Judge' in point:
            if point in text:
                return text.split(point)[-1]
        if point in text:
            return text.split(point)[1]

def split_text_tail(text):
    if pd.notnull(text):        
        split_points = ['NOTES', 'DISTRIBUTION']
        for point in split_points:
            if point in text:
                return text.split(point)[0]
        return text

long_case_text = (
    long_cases['plain_text']
          .apply(split_text_head)
          .apply(split_text_tail)
          .dropna()
          .str.strip()
)

import spacy
nlp = spacy.load('en_core_web_lg')

def preprocess(body, nlp=nlp):
    text = body.split()
    text = ' '.join(text)
    ## cut sentence fragments.
    doc = nlp(text)
    sentences = list(filter(lambda x: len(x.text.split())>5, doc.sents))
    ## 
    text = [word.text for sent in sentences for word in sent]
    body = ' '.join(text)

    ### other cleanup
    body = body.lower()
    body = body.strip()
    return body

import swifter
import util
from tqdm import tqdm_notebook as tqdm

long_case_text_processed = []
for text in tqdm(long_case_text, total=len(long_case_text)):
    long_case_text_processed.append(preprocess(text))

with open('court-processed.txt', 'w', encoding='utf-8') as f:
    for line in long_case_text_processed:
        f.write(line)
        f.write('\n')

# City Council Meeting Minutes

In [ ]:
import pandas as pd
all_meetings_df = pd.read_csv('../council_meetings/all_items_df_processed.csv', index_col=0)

def clean_num_header(text):
    numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
    first = numbers + ['-', ' ', ',']
    done = False
    while not done:
        if text[0] in first:
            text = text[1:]
        elif text[1] in numbers:
            text = text[2:]
        elif text[:2] == 'CD':
            text = text[2:]
        else:
            done = True
    return text

meeting_text = (all_meetings_df
 .loc[lambda df: df['item_num'].notnull()]['text']
#  .head()
 .str.replace(r'ITEM NO\.\(\d+\)', '')
 .str.strip()
 .apply(clean_num_header)
)

sample = meeting_text.sample(1000)

sample.to_csv('../app/data/council-meetings-for-datastore.csv', header='')
sample.to_json("../app/data/council-meetings-sample.json")

# Bills

In [8]:
bill_data = pd.read_csv('../bills/unprocessed-bills-data-all.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [16]:
bill_data['title'][5]

'"An Act relating to the procurement of supplies, services, professional services, and construction for the Alaska Energy Authority; establishing the Alaska Railbelt energy fund and relating to the fund; relating to and repealing the Railbelt energy fund; relating to the quorum of the board of the Alaska Energy Authority; relating to the powers of the Alaska Energy Authority regarding employees and the transfer of certain employees of the Alaska Industrial Development Export Authority to the Alaska Energy Authority; relating to acquiring or constructing certain projects by the Alaska Energy Authority; relating to the definition of \'feasibility study\' in the Alaska Energy Authority Act; and providing for an effective date."'

In [20]:
os.makedirs('../models/data/unlabeled-corpora')

In [21]:
t_data = open('../models/data/bert-data/train.tsv').readlines()[:100]

In [24]:
t_data[2]

'"reacting fears deadly disease may entered human food chain , european commission today imposed ban exports british beef host goods , ranging medicine cosmetics candy , ingredients beef byproducts . even though coupled offer financial aid , move , expected european veterinary committee recommended monday , rebuff brtain european partners serious blow $ 6.5 billion beef industry . action certain sour atmosphere european union negotiations open friday . [ . ] reacting fears deadly disease may entered human food chain , european commission today imposed ban exports british beef host goods , ranging medicine cosmetics candy , ingredients beef byproducts . even though coupled offer financial aid , move , expected european veterinary committee recommended monday , rebuff brtain european partners serious blow $ 6.5 billion beef industry . action certain sour atmosphere european union negotiations open friday . [ . ] also made virtually certain government , already fire handling crisis , forc